In [2]:
import pandas as pd
from xgboost import XGBRegressor
import holidays
from pathlib import Path
import numpy as np
from sklearn.preprocessing import StandardScaler

In [ ]:
target_col = 'log_bike_count'

columns_to_drop = ['coordinates', 'counter_id', 'site_id', 'counter_installation_date'] #
                   #'counter_technical_id', 'site_name']

def get_model_data(path='data/train.parquet'):

    data = pd.read_parquet(path)
    data.sort_values(['date', 'counter_name'], inplace=True)
    y = data[target_col].values
    X = data.drop(['bike_count', target_col], axis=1)
    X=data

    return X, y

In [ ]:
def covid_period(date):
    confinement_start = pd.Timestamp('2020-10-30')
    confinement_end = pd.Timestamp('2020-12-15')
    couvre_feu_1_start = pd.Timestamp('2020-12-15')
    couvre_feu_1_end = pd.Timestamp('2021-01-15')
    couvre_feu_2_start = pd.Timestamp('2021-01-16')
    couvre_feu_2_end = pd.Timestamp('2021-06-20')
    if confinement_start <= date <= confinement_end:
        return 1  # lockdown
    elif couvre_feu_1_start <= date <= couvre_feu_1_end:
        return 2  # first curfew
    elif couvre_feu_2_start <= date <= couvre_feu_2_end:
        return 3  # second curfew
    else:
        return 0

def _encode_date(date): 
    date = date.copy()
    date['month_day'] = date['date'].dt.day
    date['week_day'] = date['date'].dt.day_of_week + 1
    date['year'] = date['date'].dt.year
    date['month'] = date['date'].dt.month
    date['hour'] = date['date'].dt.hour
    years = date['year'].drop_duplicates().values.tolist()
    french_holidays = set(holidays.country_holidays('FR', years=years))
    date['is_holiday'] = (date['date']
                        .dt.date
                        .isin(french_holidays)
                        .astype(int))
    date['covid_state'] = date['date'].apply(covid_period)

    return date.drop(columns= 'date')

date_cols = ['week_day', 'year', 'month', 'hour', 'is_holiday', 'covid_state', 'month_day'] 

In [24]:
def _merge_external_data(X):
    # FONCTION UN PEU MODIFIEE PAR RAPPORT A CELLE DE LA PIPELINE
    #file_path = Path(__file__).parent / "external_data//external_data.csv"

    df_ext = pd.read_csv("external_data//external_data.csv", parse_dates=["date"])
    df_ext['date'] = pd.to_datetime(df_ext['date']).astype('datetime64[us]')

    X = X.copy()
    X["orig_index"] = np.arange(X.shape[0])
    X = pd.merge_asof(
        X.sort_values("date"), df_ext[["date", "t"]].sort_values("date"), on="date", direction='nearest',
    )
    X = X.sort_values("orig_index")
    del X["orig_index"]
    return X

In [4]:
X, y = get_model_data()
#X = _merge_external_data(X)

In [ ]:
#X['week_lag'] = X.groupby('counter_name')['log_bike_count'].shift(7).fillna(0)
#X['month_lag'] = X.groupby('counter_name')['log_bike_count'].shift(30).fillna(0)
#X.dropna(inplace=True)


C:\Users\lucie\AppData\Local\Temp\ipykernel_30272\1415321250.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  X['week_lag'] = X.groupby('counter_name')['log_bike_count'].shift(7).fillna(0)
C:\Users\lucie\AppData\Local\Temp\ipykernel_30272\1415321250.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  X['month_lag'] = X.groupby('counter_name')['log_bike_count'].shift(30).fillna(0)


In [ ]:
X.drop(columns=columns_to_drop, inplace=True)

In [6]:
X = _encode_date(X)
#cols_to_encode = X.columns[~X.columns.isin(['latitude', 'longitude'])]
#encoded_cols = pd.get_dummies(X[cols_to_encode], drop_first=True)
#non_encoded_cols = X[['latitude', 'longitude']]
#cols_to_encode = X.select_dtypes(include=['object', 'category']).columns.tolist() + date_cols
#encoded_cols = pd.get_dummies(X[cols_to_encode], drop_first=True)
#non_encoded_cols = X.loc[:, ~X.columns.isin(cols_to_encode)]

In [ ]:
#X_new = pd.concat([non_encoded_cols, encoded_cols], axis=1)

In [33]:
model = XGBRegressor(
    learning_rate=0.1,
    n_estimators=100,
    max_depth=10,
    random_state=42,
    tree_method='hist', 
    enable_categorical=True, 
    reg_lambda=6
)

In [34]:
model.fit(X, y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=True, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [24]:
# Find a way to input the lagged values from the last values of the train set into the test set 

In [35]:
X_test = pd.read_parquet('data/final_test.parquet')

#X_test['week_lag'] = X_test.groupby('counter_name')['log_bike_count'].shift(7).fillna(0)
#X_test['month_lag'] = X_test.groupby('counter_name')['log_bike_count'].shift(30).fillna(0)
X_test.drop(columns=columns_to_drop, inplace=True)

In [36]:
#X_test = _merge_external_data(X_test)
X_test = _encode_date(X_test)

In [84]:
# Performance is better without encoding...

#cols_to_encode = X_test.columns[~X_test.columns.isin(['latitude', 'longitude'])]
#encoded_cols = pd.get_dummies(X_test[cols_to_encode], drop_first=True)
#non_encoded_cols = X_test[['latitude', 'longitude']]
cols_to_encode = X_test.select_dtypes(include=['object', 'category']).columns.tolist() + date_cols
encoded_cols = pd.get_dummies(X_test[cols_to_encode], drop_first=True)
non_encoded_cols = X_test.loc[:, ~X_test.columns.isin(cols_to_encode)]
X_test_new = pd.concat([non_encoded_cols, encoded_cols], axis=1)

In [ ]:
# 10 septembre au 18 octobre

In [37]:
test_pred = model.predict(X_test)

test_df = pd.DataFrame({
    'Id': range(0, len(test_pred)), 
    'log_bike_count': test_pred           
})

test_df.to_csv('xgb_raw5.csv', index=False)

#### NEW PIPELINE ATTEMPT

In [4]:
import importlib
import bike_count as bc

In [18]:
importlib.reload(bc)

<module 'bike_count' from 'c:\\Users\\lucie\\XHEC Python for Data Science\\bike_count_ldmh\\bike_count.py'>

In [6]:
X, y = bc.get_model_data()

In [7]:
pipe = bc.xgb_table_vectorized()

In [8]:
pipe.fit(X, y)

c:\Users\lucie\anaconda3\envs\bikes-count\lib\site-packages\skrub\_check_input.py:169: UserWarning: Only pandas and polars DataFrames are supported, but input is a Numpy array. Please convert Numpy arrays to DataFrames before passing them to skrub transformers. Converting to pandas DataFrame with columns ['0', '1', …].
  warnings.warn(
c:\Users\lucie\anaconda3\envs\bikes-count\lib\site-packages\skrub\_clean_null_strings.py:39: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return col.replace(r"^\s*$", "", regex=True)
c:\Users\lucie\anaconda3\envs\bikes-count\lib\site-packages\sklearn\compose\_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to mat

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function _encode_date at 0x0000014B483F79A0>)),
                ('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('drop_cols', 'drop',
                                                  ['coordinates', 'counter_id',
                                                   'site_id',
                                                   'counter_installation_date'])])),
                ('tablevectorizer', TableVectorizer()),
                ('xgbregressor',
                 XGBRegressor(base_s...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.1,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=10, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=100, n_jobs=None,
                              num_parallel_tree=None, random_state=42, ...))])

In [9]:
test_data = pd.read_parquet("data/final_test.parquet")
test_pred = pipe.predict(test_data)

test_df = pd.DataFrame({"Id": range(0, len(test_pred)), "log_bike_count": test_pred})

test_df.to_csv("new_xgb_pipe2.csv", index=False)

c:\Users\lucie\anaconda3\envs\bikes-count\lib\site-packages\skrub\_check_input.py:169: UserWarning: Only pandas and polars DataFrames are supported, but input is a Numpy array. Please convert Numpy arrays to DataFrames before passing them to skrub transformers. Converting to pandas DataFrame with columns ['0', '1', …].
  warnings.warn(
c:\Users\lucie\anaconda3\envs\bikes-count\lib\site-packages\skrub\_clean_null_strings.py:39: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return col.replace(r"^\s*$", "", regex=True)


Pipeline: Table vectorizer, columns dropped: toutes sauf counter_installation_date, counter_name, lati, longi, date

date: date encoder perso

counter_installation_date: Classic DatetimeEncoder()

In [19]:
X, y = bc.get_model_data()

In [20]:
pipe = bc.xgb_vectorized_no_date_encoding()
pipe.fit(X, y)

c:\Users\lucie\anaconda3\envs\bikes-count\lib\site-packages\skrub\_check_input.py:169: UserWarning: Only pandas and polars DataFrames are supported, but input is a Numpy array. Please convert Numpy arrays to DataFrames before passing them to skrub transformers. Converting to pandas DataFrame with columns ['0', '1', …].
  warnings.warn(
c:\Users\lucie\anaconda3\envs\bikes-count\lib\site-packages\sklearn\compose\_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function _encode_date at 0x0000014B483F70A0>)),
                ('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('drop_cols', 'drop',
                                                  ['coordinates', 'counter_id',
                                                   'site_id', 'site_name',
                                                   'counter_technical_id'])])),
                ('tablevectorizer',
                 TableVectorizer(datetime=DatetimeEncoder(add_to...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.1,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=10, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=100, n_jobs=None,
                              num_parallel_tree=None, random_state=42, ...))])

In [21]:
test_data = pd.read_parquet("data/final_test.parquet")
test_pred = pipe.predict(test_data)

test_df = pd.DataFrame({"Id": range(0, len(test_pred)), "log_bike_count": test_pred})

test_df.to_csv("new_xgb_pipe3.csv", index=False)

c:\Users\lucie\anaconda3\envs\bikes-count\lib\site-packages\skrub\_check_input.py:169: UserWarning: Only pandas and polars DataFrames are supported, but input is a Numpy array. Please convert Numpy arrays to DataFrames before passing them to skrub transformers. Converting to pandas DataFrame with columns ['0', '1', …].
  warnings.warn(


Conclusion: je garde une RMSE autour de 0.64 en enlevant quand meme deux features catégoriques qui sont tres conséquents: site_name et counter_technical_id